In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as ss
import datetime

# Exam 1 - Coding Portion

In this section, please answer the following questions and show your work in the notebook. For each response, please check carefully what it asks, and give a response matching that. Also, please clean it up a bit so things are readable, I should be able to find what you found and follow. 

Grading breakdown:
<ul>
<li> Can I read and follow this code? 5 points. </li>
<li> Q1 - Generalize to population - 5 points. </li>
<li> Q2 - Correlation - 3 points. </li>
<li> Q3 - Multiple Groups - 5 points. </li>
<li> Q4 - Store Problem - 7 points. </li>
</ul>

Other than what's specifically instructed, you can use/import pretty much anything and take any approach you want. Just be sure to explain what you're doing and why, and make it clear what your final answers are.

## Submission

To submit, just commit and push your changes to your GitHub repository, as normal. There's no auotmated testing or anything, if yours is visible online on the repository page, you're good. 

## Data Preparation - Don't Edit This Section

In [64]:
data_uae = pd.read_csv("data/uae_real_estate_2024.csv")
data_uae = data_uae[data_uae['price'] <= 5000000]
data_uae['verified'] = data_uae['verified'].astype(bool)


data_uae["sq_ft"] = data_uae["sizeMin"].apply(lambda x: int(x.split(" ")[0].replace(",", "")) if pd.notnull(x) else np.nan)
data_uae["description_length"] = data_uae["description"].apply(lambda x: len(x) if pd.notnull(x) else 0)
data_uae["log_price"] = np.log(data_uae["price"])
data_uae.head()


data_uae.head()

C:\Users\VIP\AppData\Local\Temp\ipykernel_39728\900119103.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_uae['verified'] = data_uae['verified'].astype(bool)


,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,sq_ft,description_length,log_price
0,Great ROI I High Floor I Creek View,"Binghatti Canal, Business Bay, Dubai",3,2,2024-08-14T12:02:53Z,Residential for Sale,2500000,True,sell,1323 sqft,NO,MNA Properties is delighted to offer this apar...,1323,1272,14.731801
2,Green Belt | Corner Unit | Spacious Plot,"La Rosa 6, Villanova, Dubai Land, Dubai",3,4,2024-08-14T06:24:28Z,Residential for Sale,3600000,True,sell,2324 sqft,NO,Treo Homes is very pleased to be bringing to t...,2324,1102,15.096444
3,2BR+Study | Near Pool and Park | Private,"Springs 15, The Springs, Dubai",3,2,2024-08-15T06:07:22Z,Residential for Sale,2999999,True,sell,1647 sqft,NO,2 Bedrooms + Study | Near Pool &amp; Park | Pr...,1647,1025,14.914123
4,Vacant | Well Maintained | Area Expert,"Noor Townhouses, Town Square, Dubai",3,3,2024-08-09T08:28:59Z,Residential for Sale,2449999,True,sell,2105 sqft,NO,-Type 1\n-3 Bed+Maid\n-Close To Amenities\n-BU...,2105,1026,14.711598
6,High Floor | Motivated Seller | Full Marina View,"Vida Residences Dubai Marina, Dubai Marina, Dubai",3,2,2024-08-16T05:03:07Z,Residential for Sale,4980000,True,sell,1095 sqft,NO,Property Features:\n* 2 Bedrooms\n* Unfurnishe...,1095,1286,15.420940


In [65]:
data_uae.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3961 entries, 0 to 5057
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               3961 non-null   object 
 1   displayAddress      3961 non-null   object 
 2   bathrooms           3882 non-null   object 
 3   bedrooms            3881 non-null   object 
 4   addedOn             3961 non-null   object 
 5   type                3961 non-null   object 
 6   price               3961 non-null   int64  
 7   verified            3961 non-null   bool   
 8   priceDuration       3961 non-null   object 
 9   sizeMin             3961 non-null   object 
 10  furnishing          3959 non-null   object 
 11  description         3961 non-null   object 
 12  sq_ft               3961 non-null   int64  
 13  description_length  3961 non-null   int64  
 14  log_price           3961 non-null   float64
dtypes: bool(1), float64(1), int64(3), object(10)
memory usage: 4

## Price of Population

Our data has a sample of real estate listings in the UAE. Use this sample to estimate the mean price as well as a 95% confidence interval for the mean price of all real estate listings in the UAE.

<b>If needed, use a value of 875000 for the population count of number of homes in the UAE.</b>

In [72]:
# Generalize 

# Sample prices from  data
prices = np.array([2500000, 3600000, 2999999, 2449999])

# Sample statistics
sample_mean = np.mean(prices)
sample_std = np.std(prices, ddof=1)  # sample standard deviation
n = len(prices)
N = 875000  # population size


print(f"Sample mean price: {sample_mean:,.0f} AED")


Sample mean price: 2,887,500 AED


## Show the Correlation Between Bedrooms and Bathrooms

This may require data preparation.

<b> State your answer, anlong with a statement of what it means in reality. (e.g. if I see correlation of .02 between 'shoe size' and 'IQ', I would say that the two appear to be unrelated, and knowing one tells us nothing about the other)</b>

In [70]:


# Select and clean relevant columns
cleaned = data_uae[['bedrooms', 'bathrooms']].copy()
cleaned['bedrooms'] = pd.to_numeric(cleaned['bedrooms'], errors='coerce')
cleaned['bathrooms'] = pd.to_numeric(cleaned['bathrooms'], errors='coerce')
cleaned = cleaned.dropna()

# Extract aligned arrays
a = cleaned['bedrooms'].values
b = cleaned['bathrooms'].values

# Pearson correlation
pearson_corr = thinkstats2.Corr(a, b)
print("thinkstats2.Corr(bedrooms, bathrooms) =", round(pearson_corr, 2))

# Spearman correlation
spearman_corr = thinkstats2.SpearmanCorr(a, b)
print("thinkstats2.SpearmanCorr(bedrooms, bathrooms) =", round(spearman_corr, 2))

thinkstats2.Corr(bedrooms, bathrooms) = 0.89
thinkstats2.SpearmanCorr(bedrooms, bathrooms) = 0.88


There is a very strong positive relationship between the number of bedrooms and bathrooms in the dataset. In reality, this suggests that properties with more bedrooms tend to also have more bathrooms. Knowing the number of bedrooms gives you a reliable estimate of how many bathrooms to expect — they increase together consistently.


## Multiple Groups

Based on this data, do any of the groups (furnished, unfurnished, or partially furnished) appear to be different in terms of price per square foot? State your answer and show evidence to support your answer.

In [68]:
# Go
 #Clean 'price' and extract numeric square footage
data_uae['price'] = pd.to_numeric(data_uae['price'], errors='coerce')
data_uae['sizeMin'] = data_uae['sizeMin'].str.replace(' sqft', '', regex=False)
data_uae['sq_ft'] = pd.to_numeric(data_uae['sizeMin'], errors='coerce')

# Drop missing values
data_uae = data_uae.dropna(subset=['price', 'sq_ft', 'furnishing'])

# Standardize furnishing labels
data_uae['furnishing'] = data_uae['furnishing'].str.upper().str.strip()

# Calculate price per square foot
data_uae['price_per_sqft'] = data_uae['price'] / data_uae['sq_ft']

# Debug: check unique furnishing values
print("Furnishing groups:", data_uae['furnishing'].unique())

# Debug: check group sizes
print("Group counts:")
print(data_uae['furnishing'].value_counts())

# Create groups
group_yes = data_uae[data_uae['furnishing'] == 'YES']['price_per_sqft']
group_no = data_uae[data_uae['furnishing'] == 'NO']['price_per_sqft']
group_partial = data_uae[data_uae['furnishing'] == 'PARTLY']['price_per_sqft']

# Run ANOVA only if all groups have data

f_stat, p_value = stats.f_oneway(group_yes, group_no, group_partial)
    
p_value

Furnishing groups: ['NO' 'YES' 'PARTLY']
Group counts:
furnishing
NO        2804
YES        804
PARTLY     351
Name: count, dtype: int64


np.float64(1.3062161248095315e-30)

p<.05, The furnishing groups (furnished, unfurnished, and partially furnished) do appear to be different in terms of price per square foot.


## Store

In this scenario, we are looking to puchase a small retail store, but don't want to do so unless we project to make a profit within the first year.

This store is run by a real nerd, so they gave us some statistical data for us to estimate (the exact values are in the code cell below as variables):
<ul>
<li> The number of customers per day is described by a Poisson distribution with a rate of <b><i>customers_per_day</i></b> customers per day.</li>
<li> The buy rate (the percentage of customers that make a purchase) is <b><i>buy_rate</i></b>.</li>
<li> The amount each customer spends is described by a normal distribution with a mean of <b><i>mean_purchase_per_buyer</i></b> and a standard deviation of <b><i>std_dev_purchase_per_buyer</i></b>.</li>
<li> Each day, we need to take in at least <b><i>minimum_daily_income</i></b> to cover costs - to make it a 'good day'.</li>
<li> In a given year (we open every day), we must have over <b><i>good_day_minimum_count</i></b> good days for the year to be a success.</li>
</ul>

To determine if the store is a good investment, we will simulate the daily sales over the course of a year and see how often we meet or exceed the <b><i>minimum_daily_income</i></b> threshold. <b> Demonstrate how comfortable you are with the statement "we express this store to be a success", and show why in your calculations.</b> This is not something with one specific answer, there are many details that may vary somewhat. If it is hard, I recommend starting with a very simple model - can you simulate one customer? Can we simulate one day of customers? Can we simulate one year of days? <b>Build a simple model and make it more accurate incrementally</b>, doing it all at once can be hard, doing it piece by piece is easier. 

<b>Overall, for grades, this will be roughly logarithmicly evaluated - if you get something ok, you'll get points. The better it is the more points, but as you go up, the curve flattens. The most value is in getting a simple, working version. </b>

<b>tl;dr:</b> We have a store with customers and purchases defined by these distributions. Simulate a year of operations and deterimine the number of dayas in that year the store made over the cutoff - if it did, that's a success, if not, it's a failure. 

In [69]:
customers_per_day = 200
buy_rate = 0.22
mean_purchase_per_buyer = 125
stddev_purchase_per_buyer = 30
minimum_daily_income = 4000

good_day_minimum_rate = .10
good_day_minimum_count = 365 - (int(365 * good_day_minimum_rate))

# Get simulating!
